1. Implement Parallel Reduction using Min, Max, Sum and Average.

The 'omp' parallel sections directive effectively combines the omp parallel and omp sections directives. This directive lets you define a parallel region containing a single sections directive in one step.

In [ ]:
#include<iostream>
#include<time.h>
#include<omp.h>
#include<vector>
using namespace std;
vector<int> initialize(vector<int>,int);
int main()
{
    int SIZE = 1<<20;
    int arr[SIZE];
    vector<int> v;
    v = initialize(v,SIZE); //Function defined to initialize vectoe with random values
    int x = 0;
    vector<int>::iterator ptr;
    //Copy vector to array
    for (ptr = v.begin(); ptr < v.end(); ptr++)
    {
        arr[x]=*ptr;
	//cout<<*ptr<<endl;
        x+=1;
    }
    double start,end;
    start  =omp_get_wtime(); //Record start time
    int max_val = 0;
    int j;
	//Execute below loop parallely using omp compiler direcives
    #pragma omp parallel for reduction(max:max_val)
    for(j=0;j<SIZE;j++)
    {
        if(arr[j]>max_val)
            max_val=arr[j];
    }
    cout<<"\nMAX value from array is = "<<max_val<<endl;
    int min_val = 10000; //MAX VALUE is given as input    
    #pragma omp parallel for reduction(min:min_val)
    for(j=0;j<SIZE;j++)
    {
        if(arr[j]<min_val)
            min_val=arr[j];
    }
    cout<<"\nMIN value from array is = "<<min_val<<endl;
    int sum = 0;
    #pragma omp  parallel for reduction(+:sum)
    for(int i=0;i<SIZE;i++)
    {
        sum+=arr[i];
    }
    cout<<"\nSUM value from array is = "<<sum<<endl;
    sum = 0;
    #pragma omp  parallel for reduction(+:sum)
    for(int i=0;i<SIZE;i++)
    {
        sum+=arr[i];
    }
    int average = sum/SIZE;
    cout<<"\nAVERAGE value from array is = "<<average<<endl;
    end = omp_get_wtime();
    cout<<"Execution time = "<<end-start<<endl;
}

vector<int> initialize(vector<int> v,int n)
{
    srand(time(0));
    for(int i=0;i<n;i++)
        v.push_back(rand()%10000);
    return v;
}

In [ ]:
!g++ -fopenmp Assign1-1.c

In [ ]:
!./a.out


MAX value from array is = 9999

MIN value from array is = 0

SUM value from array is = 942541798

AVERAGE value from array is = 898
Execution time = 0.00710393


2. Write a CUDA program that, given an N-element vector, find-
-Max element of vector

-Min element of vector

-Mean of vector

-Standard deviation of vector

In [ ]:
code = """
#include<iostream>
#include<math.h>
#define n 8
using namespace std;
__global__ void minimum(int *input) 
{
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;    
   // printf("No of threads = %d", number_of_threads);
    while(number_of_threads>0) 
    {
        if(tid < number_of_threads) 
        {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}
__global__ void maximum(int *input) 
{
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;    
    while(number_of_threads>0) 
    {
        if(tid < number_of_threads) 
        {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}
__global__ void sum(int *input) 
{
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;    
    while(number_of_threads > 0) 
    {
        if(tid < number_of_threads) 
        {
            int first = tid * step_size * 2;
            int second = first + step_size;    
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}
__global__ void mean_diff_sq(float *input, float mean) 
{
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}
__global__ void sum_floats(float *input) 
{
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;    
    while(number_of_threads > 0) 
    {
        if(tid < number_of_threads) 
        {
            int first = tid * step_size * 2;
            int second = first + step_size;    
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}
void copy_int_to_float(float *dest, int *src, int size)
{
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}
void random_ints(int *input, int size) 
{
    srand((unsigned)time(0));
    for(int i=0; i<size; i++)  
    {
        input[i] = rand()%20+1;
        cout<<input[i]<<"  ";   
    }
    cout<<endl;
}
int main() 
{
    //int n=8;
    int size = n*sizeof(int); //calculate no. of bytes for array        
    int *arr;
    int *arr_d, result;
    arr = (int *)malloc(size);
    random_ints(arr, n);
    cudaMalloc((void **)&arr_d, size);
    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    minimum<<<1,n/2>>>(arr_d);
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"The minimum element is "<<result<<endl;       
    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    maximum<<<1,n/2>>>(arr_d);
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"The maximum element is "<<result<<endl;
    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    sum<<<1,n/2>>>(arr_d);
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    cout<<"The sum is "<<result<<endl;
    //AVERAGE
    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;
    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    copy_int_to_float(arr_float, arr, n);
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);
    stdValue = stdValue / n;
    cout<<"The variance is "<<stdValue<<endl;
    stdValue = sqrt(stdValue);
    cout<<"The standard deviation is "<<stdValue<<endl;
    cudaFree(arr_d);
    return 0;
}
"""

In [ ]:
t=open("Assign1.cu","w")
t.write(code)
t.close()


In [ ]:
!nvcc Assign1.cu

In [ ]:
!./a.out

17  7  11  10  5  14  13  14  
The minimum element is 5
The maximum element is 17
The sum is 91
The mean is 11.375
The variance is 13.7344
The standard deviation is 3.70599


In [ ]:
!nvprof ./a.out

17  16  2  9  8  7  8  5  
==2482== NVPROF is profiling process 2482, command: ./a.out
The minimum element is 2
The maximum element is 17
The sum is 72
The mean is 9
The variance is 23
The standard deviation is 4.79583
==2482== Profiling application: ./a.out
==2482== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   23.90%  8.8960us         4  2.2240us  2.0480us  2.3680us  [CUDA memcpy DtoH]
                   18.40%  6.8480us         4  1.7120us  1.5040us  2.2720us  [CUDA memcpy HtoD]
                   13.59%  5.0560us         1  5.0560us  5.0560us  5.0560us  minimum(int*)
                   12.12%  4.5120us         1  4.5120us  4.5120us  4.5120us  sum_floats(float*)
                   11.69%  4.3520us         1  4.3520us  4.3520us  4.3520us  maximum(int*)
                   11.35%  4.2240us         1  4.2240us  4.2240us  4.2240us  sum(int*)
                    8.94%  3.3280us         1  3.3280us  3.3280us  3.3280us

In [ ]:
code2="""
#include <bits/stdc++.h> 
#include<math.h>
#include <chrono>
using namespace std; 
using namespace std::chrono; 
int main() 
{ 
  auto start = high_resolution_clock::now();
    vector<int> a; 
    cout << "Vector: "; 
    srand((unsigned)time(0));
    for(int i = 0; i < 8; i++) 
    {
      a.push_back(rand()%20+1);
    }     
    cout << endl; 
    for (auto i = a.begin(); i != a.end(); ++i) 
        cout << *i << " ";
    cout<<endl <<"Max:"<< *min_element(a.begin(), a.end()) << endl; 
    cout <<"Min:"<< *max_element(a.begin(), a.end()) << endl; 
    double sum;
    for (auto i = a.begin(); i != a.end(); ++i)
        sum=sum+ (*i);
    cout <<"Sum:"<<sum<<endl;
    long mean=sum/a.size();
    cout<<"Mean:"<<mean<<endl;
    double var;
    for (auto i = a.begin(); i != a.end(); ++i)
        var=var + ((*i-mean) * (*i-mean));
    var/=8;
    cout<<"Variance:"<<var<<endl;
    cout<<"Standard Deviation:"<<sqrt(var)<<endl;
    auto stop = high_resolution_clock::now(); 
    auto duration = duration_cast<microseconds>(stop - start);  
    cout <<"Execution Time:"<< duration.count() << " microseconds" << endl;
    return 0; 
}  """

In [ ]:
text=open("a.cpp","w")
text.write(code2)
text.close()

In [ ]:
!g++ a.cpp

In [ ]:
!./a.out

Vector: 
1 8 15 7 10 14 17 11 
Max:1
Min:17
Sum:83
Mean:10
Variance:23.125
Standard Deviation:4.80885
Execution Time:96 microseconds
